In [ ]:
# crack 2.17

In [ ]:
!pip install pytorch-lightning

In [ ]:
import pandas as pd
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import time
import torch.nn as nn
import pytorch_lightning as L

torch.manual_seed(2147483647)

In [95]:
names = pd.read_csv("../names.txt",names = ["label"])

In [ ]:
all_stoi = {}
all_itos ={}

all = set("".join(names.label+"."))
for index,(chr) in enumerate(sorted(all)):
    all_stoi[chr] = index

for index,(chr) in enumerate(sorted(all)):
    all_itos[index] = chr

In [ ]:
x = []
y = []
blocks = 3

for word in names.label:
    wx = [("."* (blocks - i)) + word  for i in range(blocks)]
    wy = [y.append(all_stoi[i]) for i in (word+".")]
    for ch1,ch2,ch3 in zip(*wx[:len(wx)]):
        x.append((all_stoi[ch1],all_stoi[ch2],all_stoi[ch3]))

In [ ]:
device = torch.device("mps" if torch.mps.is_available() else "cpu")

In [ ]:
# train: 70%
# val: 10%
# test: 30%

train_x,val_x,train_y,val_y = train_test_split(x,y,test_size=0.2, shuffle=True,stratify = y, random_state=42)
val_x,test_x,val_y,test_y = train_test_split(val_x,val_y,test_size=0.1, shuffle=True,stratify = val_y, random_state=42)



train_x = torch.tensor(train_x,device = device)
train_y = torch.tensor(train_y,device = device)

val_x = torch.tensor(val_x,device = device)
val_y = torch.tensor(val_y,device = device)

test_x = torch.tensor(test_x,device = device)
test_y = torch.tensor(test_y,device = device)

# print(f"x: {x.shape} y: {y.shape}")
print(f"train_x: {train_x.shape} train_y: {train_y.shape}")
print(f"val_x: {val_x.shape} val_y: {val_y.shape}")
print(f"test_x: {test_x.shape} test_y: {test_y.shape}")

In [ ]:
# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        super().__init__()
        self.data = data.to(device)
        self.labels = labels.to(device)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
       
        x =  self.data[idx]
        y =  self.labels[idx]
        return x,y

        
train_ds = CustomDataset(train_x,train_y)
val_ds = CustomDataset(val_x,val_y)
test_ds = CustomDataset(test_x,test_y)

train_loader = DataLoader(train_ds,batch_size=64,shuffle=True)
test_loader = DataLoader(test_ds,batch_size=32,shuffle=True)
val_loader = DataLoader(val_ds,batch_size=32,shuffle=False)


In [ ]:
# parameter
c = torch.randn(27,2,device =device)

w = torch.randn(6,100,device =device)
b = torch.randn(100,device =device)


w1 = torch.randn(100,27,device =device)
b1 = torch.randn(27,device =device)

lr = 0.01 #loss=tensor(2.0388, grad_fn=<NllLossBackward0>)
# lr = 0.1

parameter = [c,w,b,w1,b1]

for p in parameter:
    p.requires_grad =True


optimizer = torch.optim.SGD(parameter,lr)

In [ ]:
# lossi = []
# epochi = []

In [ ]:
start = time.time()
for epoch in range(10000):
    # Forward Pass
    # for index,(feature,label) in enumerate(train_loader):
        emb = c[train_x] # [32, 3, 2]
        z = torch.tanh(emb.view(-1,w.shape[0]) @ w + b)
        logits = z @ w1 + b1
        loss = F.cross_entropy(logits,train_y)
        # lossi.append(loss.item())
        # epochi.append(epoch)
        
        ##Backward Pass
        # for p in parameter:
        #     p.grad = None
        # loss.backward()
        # p.data += -lr*p.grad
        ## same as
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
end = time.time()
delay = end - start
print(f"time: {delay*60}min: {loss.item()=}")   

In [ ]:
plt.scatter(lossi,epochi)

plt.grid("minor")
plt.show()

In [ ]:
# 16.4423

In [ ]:
# Training loss
for index,(feature,label) in enumerate(train_loader):
    emb = c[feature] # [32, 3, 2]
    z = torch.tanh(emb.view(-1,w.shape[0]) @ w + b)
    logits = z @ w1 + b1
    loss = F.cross_entropy(logits,label)

print(f"{loss.item()=}")  

In [ ]:
# Val loss
for index,(feature,label) in enumerate(val_loader):
    emb = c[feature] # [32, 3, 2]
    z = torch.tanh(emb.view(-1,w.shape[0]) @ w + b)
    logits = z @ w1 + b1
    loss = F.cross_entropy(logits,label)

print(f"{loss.item()=}")  

# Automate training with Lightning

In [100]:

# Model
class MLnames(nn.Module):
  def __init__(self):
    super().__init__()
    # Initialize parameters
    self.c = nn.Parameter(torch.randn(27,2,device =device))
    self.w = nn.Parameter(torch.randn(6,100,device =device))
    self.b = nn.Parameter(torch.randn(100,device =device))
    
    self.w1 = nn.Parameter(torch.randn(100,27,device =device))
    self.b1 = nn.Parameter(torch.randn(27,device =device))

    # self.all_layer = nn.Sequential(
    #   # Input Layer
    #     nn.Linear(6,100),
    #     nn.Tanh(),

    #   # Hidden Layer
    #     nn.Linear(100,100),
    #     nn.Tanh(),

    #   # Output Layer
    #     nn.Linear(100,27)
    # )
    
  # Forward Pass
  def forward(self,x):
    emb = self.c[x] # [32, 3, 2]
    z = torch.tanh(emb.view(-1,self.w.shape[0]) @ w + b)
    logits = self.z @ self.w1 + self.b1
    return logits
      



In [101]:
# Execute the model with PyTorch Lightning
class LightningModelExe(L.LightningModule):
    def __init__(self,model,lr):
        super().__init__()
        self.model = model
        self.lr = lr

    def forward(self, inputs):
        return self.model(inputs)

    def training_step(self, batch, batch_idx):
        x,y = batch
        output = self.forward(x)
        loss = F.cross_entropy(output, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x,y = batch
        output = self.forward(x)
        loss = F.cross_entropy(output, y)
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        x,y = batch
        output = self.forward(x)
        loss = F.cross_entropy(output, y)
        self.log("test_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.SGD(self.model.parameters(), lr=self.lr)

In [102]:
xo = []
yo = []
blocks = 3
for word in names.label:
    wx = [("."* (blocks - i)) + word  for i in range(blocks)]
    wy = [yo.append(all_stoi[i]) for i in (word+".")]
                       
    for ch1,ch2,ch3 in zip(*wx[:len(wx)]):
        xo.append((all_stoi[ch1],all_stoi[ch2],all_stoi[ch3]))
 
train_x,val_x,train_y,val_y = train_test_split(xo,yo,test_size=0.2,stratify = y,random_state=42)
val_x,test_x,val_y,test_y = train_test_split(val_x,val_y,test_size=0.1,stratify = val_y, random_state=42)



train_x = torch.tensor(train_x,device = device)
train_y = torch.tensor(train_y,device = device)

val_x = torch.tensor(val_x,device = device)
val_y = torch.tensor(val_y,device = device)

test_x = torch.tensor(test_x,device = device)
test_y = torch.tensor(test_y,device = device)


train_ = {"train_x": train_x, "train_y": train_y}
valid_ = {"val_x": val_x, "val_y": val_y}
test_ = {"test_x": test_x, "test_y": test_y}



print(f"train_x: {len(train_x)} train_y: {len(train_y)}")
print(f"val_x: {len(val_x)} val_y: {len(val_y)}")
print(f"test_x: {len(test_x)} test_y: {len(test_y)}")

# print(len(xo),len(yo))

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        super().__init__()
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x =  self.data[idx]
        y =  self.labels[idx]
        return x,y
        
# # Execute the Data Loading with PyTorch Lightning
class DataModule(L.LightningDataModule):
    def __init__(self, data_dir: str = "../names.txt", batch_size: int = 32):
        super().__init__()
        self.data_dir = pd.read_csv(data_dir,names = ["label"])
        self.batch_size = batch_size

    def setup(self, stage: str = None,train = train_,valid = valid_,test = test_):
        if stage == "fit" or stage is None:
            self.train_ds = CustomDataset(train["train_x"], train["train_y"])
            self.val_ds = CustomDataset(valid["val_x"], valid["val_y"])
        if stage == "validate" or stage is None:
            self.val_ds = CustomDataset(valid["val_x"], valid["val_y"])
        if stage == "test" or stage is None:
            self.test_ds = CustomDataset(test["test_x"], test["test_y"])

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size,shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.batch_size,shuffle=True)


train_x: 182516 train_y: 182516
val_x: 41067 val_y: 41067
test_x: 4563 test_y: 4563


In [103]:
if __name__ == "__main__":
    model = MLnames()
    lightning_model = LightningModelExe(model, lr=0.01)
    data_module = DataModule(data_dir="../names.txt", batch_size=32)
    trainer = L.Trainer(max_epochs=10)
    trainer.fit(lightning_model, datamodule=data_module)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type    | Params | Mode 
------------------------------------------
0 | model | MLnames | 3.5 K  | train
------------------------------------------
3.5 K     Trainable params
0         Non-trainable params
3.5 K     Total params
0.014     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.13/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


RuntimeError: Tensor for argument #2 'mat2' is on CPU, but expected it to be on GPU (while checking arguments for mm)